In [1]:
import numpy as np
import pandas as pd
from src.utils.filter_conditions_monthly import filter_conditions_monthly

EUROSTAT_PATH = "src/data/raw/eurostat/latest_data.csv"
BNETZA_PATH = "src/data/raw/germany_household/latest_data.csv"
CBS_PATH = "src/data/raw/CBS_dutch_power.csv"

In [7]:
# 1. Load and preprocess daily demand data
df = pd.read_csv("src/data/processed/daily_demand_all.csv")
df['date'] = pd.to_datetime(df['date'].astype(str).str[:10])
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

# Group by relevant columns and convert to TWh
df = df.groupby(['country', 'type', 'source', 'year', 'month'], as_index=False).agg({'demand': 'sum'})
df['demand'] = df['demand'] / 1000000000

In [8]:
# 2. Load and process Eurostat data
eurostat_df = pd.read_csv(EUROSTAT_PATH)
eurostat_df['date'] = pd.to_datetime(eurostat_df['date'])
eurostat_df['month'] = eurostat_df['date'].dt.month
eurostat_df['year'] = eurostat_df['date'].dt.year
del eurostat_df['date']
df = pd.concat([df, eurostat_df], ignore_index=True)

In [9]:
# 3. Load and process BNetzA data 
bnetza_df = pd.read_csv(BNETZA_PATH)
bnetza_df['date'] = pd.to_datetime(bnetza_df['date'])
bnetza_df['month'] = bnetza_df['date'].dt.month
bnetza_df['year'] = bnetza_df['date'].dt.year
bnetza_append = bnetza_df[['country', 'type', 'source', 'year', 'month', 'demand']]
df = pd.concat([df, bnetza_append], ignore_index=True)

In [10]:
# 4. Load and process CBS data
# we use historic CBS power data (pre July 2022) for power values and converting industry-power to industry
cbs_df = pd.read_csv(CBS_PATH)

#set the month and year columns as datetypes for each
cbs_df['month'] = pd.to_datetime(cbs_df['month'], format="%m").dt.month
cbs_df['year'] = pd.to_datetime(cbs_df['year'], format="%Y").dt.year

#isolate NL and 'industry-power' rows
df_nl_temp = df[(df['country'] == 'NL') & (df['type'] == 'industry')].copy()

#adjust older industry-power rows to industry (by subtracting power)
df_merge = df_nl_temp.merge(cbs_df[['demand','year','month']], on=['year','month'],how='left')
df_merge.replace(np.nan,0,inplace=True)
df_merge['demand'] = df_merge['demand_x'] - df_merge['demand_y']
df_merge['source'] = "entsog, CBS"

df_merge = df_merge[['country', 'type', 'source', 'year', 'month', 'demand']].copy()
cbs_df = cbs_df[['country', 'type', 'source', 'year', 'month', 'demand']].copy()

df = pd.concat([df, df_merge], ignore_index=True)
df = pd.concat([df, cbs_df], ignore_index=True)

In [11]:
# 5. Apply filtering conditions

conditions_df = pd.DataFrame(filter_conditions_monthly, columns=['country', 'type', 'source'])
filtered_df = df.merge(conditions_df, on=['country', 'type', 'source'])

print("\nAfter applying filters:")
display(filtered_df.head())


After applying filters:


,country,type,source,year,month,demand
0,AT,power,energy-charts,2018,12,0.002566
1,AT,power,energy-charts,2019,1,3.147638
2,AT,power,energy-charts,2019,2,2.488796
3,AT,power,energy-charts,2019,3,1.331403
4,AT,power,energy-charts,2019,4,1.055020


In [12]:
# 6. Aggregate data
aggregated_df = (
    filtered_df.groupby(['country', 'type', 'year', 'month'], as_index=False)
    .agg({
        'demand': 'sum',
        'source': lambda x: ', '.join(sorted(set(x))) if len(set(x)) > 1 else x.iloc[0]
    })
)

In [13]:
def track_available_demand(df):
    pivot_df = df.pivot_table(index=["country", "year", "month"], columns="type", values="demand", aggfunc="sum")
    
    records = []
    
    for index, row in pivot_df.iterrows():
        country, year, month = index
        existing_types = set(row.dropna().index)
        
        records.append({
            "country": country,
            "year": year,
            "month": month,
            "available_types": list(existing_types)
        })
    
    tracking_df = pd.DataFrame(records)
    return tracking_df


def adjust_demand(aggregated_df, new_type, operation, required_types_in=None, required_types_not_in=None):
    """
    param: 
    return:
    """
    tracking_available_demand = track_available_demand(aggregated_df)
    
    required_types_in = required_types_in if required_types_in is not None else set()
    required_types_not_in = required_types_not_in if required_types_not_in is not None else set()
    
    valid_entries = tracking_available_demand[
        tracking_available_demand['available_types'].apply(
            lambda x: required_types_in.issubset(set(x)) and required_types_not_in.isdisjoint(set(x))
        )
    ]
    
    new_rows = []
    modified_entries = set()
    
    for _, row in valid_entries.iterrows():
        country, year, month = row['country'], row['year'], row['month']
        subset = aggregated_df[(aggregated_df['country'] == country) &
                               (aggregated_df['year'] == year) &
                               (aggregated_df['month'] == month)]
        
        new_demand = operation(subset)
        if new_demand is not None:
            new_rows.append({
                'country': country,
                'year': year,
                'month': month,
                'type': new_type,
                'demand': new_demand,
                'source': 'calculated'
            })
            modified_entries.add((country, year, month))  # Track modified entries
    
    if new_rows:
        new_df = pd.DataFrame(new_rows)
        aggregated_df = pd.concat([aggregated_df, new_df], ignore_index=True)
    
    return aggregated_df, modified_entries



def industry_str_demand_operation(subset):
    industry_power_demand = subset[subset['type'] == 'industry-power']['demand'].values
    power_demand = subset[subset['type'] == 'power']['demand'].values
    return industry_power_demand[0] - power_demand[0] if len(industry_power_demand) > 0 and len(power_demand) > 0 else None


def industry_sub_demand_operation(subset):
    total_demand = subset[subset['type'] == 'total']['demand'].values
    power_demand = subset[subset['type'] == 'power']['demand'].values
    household_demand = subset[subset['type'] == 'household']['demand'].values
    return total_demand[0] - power_demand[0] - household_demand[0] if len(total_demand) > 0 and len(power_demand) > 0 and len(household_demand) > 0 else None


def total_demand_operation(subset):
    return subset['demand'].sum()


def industry_household_demand_operation(subset):
    total_demand = subset[subset['type'] == 'total']['demand'].values
    power_demand = subset[subset['type'] == 'power']['demand'].values
    return total_demand[0] - power_demand[0] if len(total_demand) > 0 and len(power_demand) > 0 else None

# Apply operations dynamically based on missing data
updated_aggregated_df, modified_industry_str = adjust_demand(
    aggregated_df, 'industry', industry_sub_demand_operation, 
    {'total', 'power', 'household'}, {'industry'}
)

updated_aggregated_df, modified_industry_str = adjust_demand(
    updated_aggregated_df, 'industry', industry_str_demand_operation, 
    {'industry-power', 'power', 'household'}, {'industry'}
)

# Remove 'industry-power' only for modified entries
updated_aggregated_df = updated_aggregated_df[
    ~((updated_aggregated_df['type'] == 'industry-power') & 
      (updated_aggregated_df[['country', 'year', 'month']].apply(tuple, axis=1).isin(modified_industry_str))
    )
].reset_index(drop=True)

updated_aggregated_df, _ = adjust_demand(
    updated_aggregated_df, 'total', total_demand_operation, None, {'total'}
)


# Remove 'industry' only for modified entries
final_aggregated_df, modified_industry_household = adjust_demand(
    updated_aggregated_df, 'industry-household', industry_household_demand_operation, 
    {'power', 'total'}, {'industry', 'household'}
)


# Recalculate tracking_available_demand
tracking_available_demand = track_available_demand(final_aggregated_df)



In [14]:
# Remove unwanted countries
final_aggregated_df = final_aggregated_df[~final_aggregated_df['country'].isin(["EU27_2020", "CY", "MT"])]

def compute_country_contributions(group):
    total_demand = group['demand'].sum()  # Compute total demand for the group
    
    contributions = {
        country: round(float(group[group['country'] == country]['demand'].sum() / total_demand) * 100, 10)  
        for country in group['country'].unique()
    }
    
    return pd.Series({'country_contributions': contributions, 'demand': total_demand})  # Rename total_demand to demand

# Group by type, year, and month and compute demand + country-wise contributions
df_eu = final_aggregated_df[final_aggregated_df['country'] != "UK"].copy()
df_pivot = df_eu.groupby(['type', 'year', 'month']).apply(compute_country_contributions).reset_index()

# Assign 'EU' as the country
df_pivot['country'] = 'EU'

# Add source column
df_pivot['source'] = 'calculated'

# Convert country contributions to a properly formatted dictionary with float values
df_pivot['country_contributions'] = df_pivot['country_contributions'].apply(lambda x: {k: float(v) for k, v in x.items()})

# Concatenate with the original dataset
final_aggregated_df = pd.concat([final_aggregated_df, df_pivot], ignore_index=True)

C:\Users\marie.juge\AppData\Local\Temp\ipykernel_25660\551365456.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_pivot = df_eu.groupby(['type', 'year', 'month']).apply(compute_country_contributions).reset_index()


In [15]:
final_aggregated_df.tail()

,country,type,year,month,demand,source,country_contributions
7236,EU,total,2024,9,200.044867,calculated,"{'AT': 2.0272649035, 'BG': 0.9477279794, 'CZ':..."
7237,EU,total,2024,10,254.417205,calculated,"{'AT': 2.2783919677, 'BG': 0.8652275142, 'CZ':..."
7238,EU,total,2024,11,379.520321,calculated,"{'AT': 2.7016817053, 'BG': 0.8001083239, 'CZ':..."
7239,EU,total,2024,12,434.119339,calculated,"{'AT': 2.480170236, 'BG': 0.7089704806, 'CZ': ..."
7240,EU,total,2025,1,265.395104,calculated,"{'AT': 2.7722711861, 'BG': 0.7213714483, 'CZ':..."


In [16]:
# 9. Final processing and export
final_aggregated_df = final_aggregated_df[final_aggregated_df['year'] >= 2019]
final_aggregated_df['demand'] = final_aggregated_df['demand'].round(2)

#### filter until 2025 - can be cleaner
final_aggregated_df = final_aggregated_df[final_aggregated_df['year'] < 2025]


In [17]:
# Save the results
final_aggregated_df.to_csv("src/data/analyzed/monthly_demand_clean.csv", index=False)
final_aggregated_df.to_json("src/data/analyzed/monthly_demand_clean.json", orient='records', indent=4)
with pd.ExcelWriter("src/data/analyzed/monthly_demand_clean.xlsx", engine='openpyxl') as writer:
    final_aggregated_df.to_excel(writer, sheet_name="Aggregated Data", index=False)


# add output to to_xlsx 

In [18]:
#### A CHECK THAT EU NUMBERS MAKE SENSE

df_check_eu = final_aggregated_df[(final_aggregated_df['country'] == 'EU')].pivot_table(index=['year','month'],columns='type',values='demand')
df_check_eu['total_calculated'] = df_check_eu[['household','industry','industry-household','power']].sum(axis=1)
df_check_eu

type        household  industry  industry-household  power   total  \
year month                                                           
2019 1         267.86     99.82               75.46  83.36  526.50   
     2         195.83     86.41               58.12  63.36  403.72   
     3         164.99     88.39               52.11  55.49  360.99   
     4         108.00     79.94               41.70  51.12  280.77   
     5          85.55     80.51               37.41  49.85  253.33   
...               ...       ...                 ...    ...     ...   
2024 8          30.10     53.33               39.45  53.88  176.75   
     9          44.30     60.84               45.15  49.75  200.04   
     10         78.96     66.07               57.56  51.83  254.42   
     11        152.52     72.74               75.44  78.82  379.52   
     12        197.09     75.91               84.23  76.89  434.12   

type        total_calculated  
year month                    
2019 1                526.50  
     2                403.72  
     3                360.98  
     4                280.76  
     5                253.32  
...                      ...  
2024 8                176.76  
     9                200.04  
     10               254.42  
     11               379.52  
     12               434.12  

[72 rows x 6 columns]